In [1]:
def load_data():
    train_data = open("train.csv").read()
    train_data = train_data.split("\n")[1:-1]
    train_data = [i.split(",") for i in train_data]
    # print(len(train_data))
    X_train = np.array([[int(i[j]) for j in range(1,len(i))] for i in train_data])
    y_train = np.array([int(i[0]) for i in train_data])

    # print(X_train.shape, y_train.shape)

    test_data = open("test.csv").read()
    test_data = test_data.split("\n")[1:-1]
    test_data = [i.split(",") for i in test_data]
    # print(len(test_data))
    X_test = np.array([[int(i[j]) for j in range(0,len(i))] for i in test_data])

    # print(X_test.shape)

    return X_train, y_train, X_test

In [2]:
import numpy as np
X_train, y_train, X_test = load_data()

In [4]:
print X_train.shape, y_train.shape, X_test.shape

(42000L, 784L) (42000L,) (28000L, 784L)


In [7]:
W = np.random.rand(784, 10) * 0.001

In [8]:
def svm_loss_vectorized(W, X, y):
    d, C = W.shape 
    _, N = X.shape 
    loss = 0 
    reg = .1
    dW = np.zeros_like(W)
    
    Z = W.T.dot(X)     
    
    correct_class_score = np.choose(y, Z).reshape(N,1).T     
    margins = np.maximum(0, 1 + Z - correct_class_score) 
    margins[y, np.arange(margins.shape[1])] = 0
    loss = np.sum(margins, axis = (0, 1))
    loss /= N 
    loss += 0.5 * reg * np.sum(W * W)
    
    F = (margins > 0).astype(int)
    F[y, np.arange(F.shape[1])] = np.sum(-F, axis = 0)
    dW = X.dot(F.T)/N + reg*W
    return loss, dW

def multiclass_svm_GD(X, y, Winit):         
    batch_size = 1000
    num_iters = 500
    print_every = 50
    reg = .001
    lr=.0000001
    bestW = Winit 
    loss_history = np.zeros((num_iters))
    bestloss = ("float")
    for it in range(num_iters):
        # randomly pick a batch of X
        idx = np.random.choice(X.shape[1], batch_size)
        X_batch = X[:, idx]
        y_batch = y[idx]
        loss_history[it], dW = svm_loss_vectorized(Winit, X_batch, y_batch)
        loss=loss_history[it]
        Winit -= lr*dW 
        if loss < bestloss: # keep track of the best solution
            bestloss = loss
            bestW = Winit
        if it % print_every == 0:
            print 'it %d/%d, loss = %.2f' \
                %(it, num_iters, loss_history[it]) 

    return bestW, bestloss
        
W,bestloss = multiclass_svm_GD(Xtr, y_train, W)
print 'bestloss: ',bestloss

C:\Users\Admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: unorderable dtypes; returning scalar but in the future this will be an error


it 0/500, loss = 9.69
it 50/500, loss = 1.84
it 100/500, loss = 1.32
it 150/500, loss = 1.15
it 200/500, loss = 0.98
it 250/500, loss = 0.86
it 300/500, loss = 0.89
it 350/500, loss = 0.80
it 400/500, loss = 0.79
it 450/500, loss = 0.70
bestloss:  0.5598351043687061


In [9]:
scores = W.T.dot(X_test.T)
Ypre = np.argmax(scores, axis = 0)
print Ypre

[2 0 9 ... 3 9 2]


In [10]:
import pandas as pd
id=np.arange(1,28001)
raw_data = {'ImageId': id, 'Label': Ypre }
df = pd.DataFrame(raw_data, columns = ['ImageId', 'Label'])
df.to_csv('sample.csv')
